In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("E:\\1currently working\\imdb_movie_review_sentiment-main\\dataset\\IMDB Dataset.csv")

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.shape

(50000, 2)

In [5]:
type(data)

pandas.core.frame.DataFrame

In [6]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [7]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [8]:
# one hot encoding
# label encoder

In [9]:
# positive -> 1
# negative -> 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [12]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [13]:
# LSTM -> LONG SHORT TERM MEMORY
# RNN -> TEXTUAL DATA

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [16]:
train_data.shape

(40000, 2)

In [17]:
test_data.shape

(10000, 2)

In [18]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [19]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [20]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]])

In [21]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]])

In [22]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [23]:
Y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [24]:
# LSTM MODEL BUILDING

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

model = Sequential()
# Use input_shape to specify the input dimension (200,) instead of input_length
model.add(Embedding(input_dim=5000, output_dim=128, input_shape=(200,)))
# The input shape for LSTM will be inferred from the Embedding layer
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.summary()

In [30]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [32]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 218s 426ms/step - accuracy: 0.7107 - loss: 0.5460 - val_accuracy: 0.8380 - val_loss: 0.3785
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 183s 267ms/step - accuracy: 0.8451 - loss: 0.3656 - val_accuracy: 0.8577 - val_loss: 0.3384
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 137s 274ms/step - accuracy: 0.8690 - loss: 0.3224 - val_accuracy: 0.8574 - val_loss: 0.3403
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 146s 292ms/step - accuracy: 0.8835 - loss: 0.2816 - val_accuracy: 0.7606 - val_loss: 0.4889
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 270ms/step - accuracy: 0.8515 - loss: 0.3443 - val_accuracy: 0.8712 - val_loss: 0.3226


In [33]:
loss, accuracy=model.evaluate(X_test, Y_test)
print(f"Test loss {loss}")
print(f"Test accuracy {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step - accuracy: 0.8681 - loss: 0.3226
Test loss 0.3179793953895569
Test accuracy 0.8702999949455261


In [34]:
model.save("senti-model.h5")
# Save the model in the recommended Keras format
#model.save('my_model.keras')


In [35]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [88]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.8728 - loss: 0.3174


In [89]:
print(loss)

0.3178520202636719


In [90]:
print(accuracy)

0.8718000054359436


In [91]:
# Building Predictive System

In [92]:
def predictive_system(review):
    # Convert the input text to a sequence of integers
    sequences = tokenizer.texts_to_sequences([review])
    
    # Pad the sequence to ensure it matches the input length required by the model
    padded_sequence = pad_sequences(sequences, maxlen=200)
    
    # Make a prediction using the trained model
    prediction = model.predict(padded_sequence)

    # Convert the prediction probability to a sentiment label
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    
    return sentiment


In [93]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step


'positive'

In [94]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'positive'

In [95]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


'positive'

In [96]:
predictive_system("i didnt like it")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


'positive'